In [1]:
import pandas as pd

Primero, compruebo la ruta de importación y cómo hacerlo para un .dat

In [71]:
import pandas as pd
import os

# 1. Definimos la ruta de la carpeta (fíjate en la 'r' al principio)
ruta_carpeta = r'/home/jovyan/work/datasets/Originales'

# 2. Elige el archivo que quieres ver (ej: ratings.csv, movies.csv)
nombre_archivo = 'users.dat' 

# 3. Creamos la ruta completa uniendo carpeta y archivo
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)

# 4. Cargamos SOLO las primeras 5 filas
df = pd.read_csv(ruta_completa, sep='::', engine='python', header=None, nrows=5)

# 5. Mostramos el resultado
print("Columnas:", df.columns.tolist())
print("-" * 30)
print(df)

Columnas: [0, 1, 2, 3, 4]
------------------------------
   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7   2460
4  5  M  25  20  55455


Luego, compruebo el resto

In [3]:
for i in ["movies","ratings","users"]:
    ruta_completa = os.path.join(ruta_carpeta, f"{i}.dat")
    df = pd.read_csv(ruta_completa, sep='::', engine='python', header=None, encoding='latin-1', nrows=5)
    print(f"Archivo: {i}.csv")
    print("Columnas:", df.columns.tolist())
    print("-" * 30)
    print(df)
    print("\n")


Archivo: movies.csv
Columnas: [0, 1, 2]
------------------------------
   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy


Archivo: ratings.csv
Columnas: [0, 1, 2, 3]
------------------------------
   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291


Archivo: users.csv
Columnas: [0, 1, 2, 3, 4]
------------------------------
   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7   2460
4  5  M  25  20  55455




# Exploración y transformación de datos

## Movies

In [20]:
nombre_archivo = 'movies.dat' 
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)

df_movies = pd.read_csv(ruta_completa, sep='::', engine='python', header=None, names=['filmId', 'film', 'genders'], encoding='latin-1')
print(len(df_movies))


3883


In [21]:
df_movies

,filmId,film,genders
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [39]:
df_movies.describe(include="all")

,filmId,film,genders
count,3883.000000,3883,3883
unique,NaN,3883,301
top,NaN,Toy Story (1995),Drama
freq,NaN,1,843
mean,1986.049446,NaN,NaN
std,1146.778349,NaN,NaN
min,1.000000,NaN,NaN
25%,982.500000,NaN,NaN
50%,2010.000000,NaN,NaN
75%,2980.500000,NaN,NaN


Comprobado, no hay nulos. Además, los valores máximo y mínimo de filmId tienen sentido (no hay outliers) y la máxima frecuencia de una película es 1 (no se repiten nombres). Aún queda comprobar los géneros

Vamos a transformar un poco los datos

In [22]:
# Limpieza: año desde el título, edad y géneros como lista
df_movies_mod = df_movies.copy()
df_movies_mod['year'] = pd.to_numeric(df_movies_mod['film'].str.extract(r'\((\d{4})\)')[0], errors='coerce')
df_movies_mod['film'] = df_movies_mod['film'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
to_list = lambda g: [] if pd.isna(g) else [part for part in g.split('|') if part]
df_movies_mod['genders'] = df_movies_mod['genders'].apply(to_list)
len(df_movies_mod)


3883

In [23]:
df_movies_mod

,filmId,film,genders,year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
3878,3948,Meet the Parents,[Comedy],2000
3879,3949,Requiem for a Dream,[Drama],2000
3880,3950,Tigerland,[Drama],2000
3881,3951,Two Family House,[Drama],2000


In [54]:
# Conteo de géneros únicos
df_movies_mod['genders'].explode().value_counts()

genders
Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Fantasy          68
Western          68
Film-Noir        44
Name: count, dtype: int64

No hay nada extraño en géneros así que lo doy por bueno

## Users

In [31]:
nombre_archivo = 'users.dat' 
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)

df_users = pd.read_csv(ruta_completa, sep='::', engine='python', header=None, names=['userId', 'gender', 'age', 'occupation', 'zip'], encoding='latin-1')
print(len(df_users))

6040


In [32]:
df_users

,userId,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [40]:
df_users.describe(include="all")

,userId,gender,age,occupation,zip
count,6040.000000,6040,6040.000000,6040.000000,6040
unique,NaN,2,NaN,NaN,3439
top,NaN,M,NaN,NaN,48104
freq,NaN,4331,NaN,NaN,19
mean,3020.500000,NaN,30.639238,8.146854,NaN
std,1743.742145,NaN,12.895962,6.329511,NaN
min,1.000000,NaN,1.000000,0.000000,NaN
25%,1510.750000,NaN,25.000000,3.000000,NaN
50%,3020.500000,NaN,25.000000,7.000000,NaN
75%,4530.250000,NaN,35.000000,14.000000,NaN


In [58]:
# Transformaciones sobre df_users
occupation_map = {
    0: 'Otro / No especificado', 1: 'Académico / Educador', 2: 'Artista',
    3: 'Administrativo / Oficina', 4: 'Estudiante universitario / Postgrado',
    5: 'Atención al cliente', 6: 'Médico / Sector salud', 7: 'Ejecutivo / Gerente',
    8: 'Agricultor', 9: 'Amo/a de casa', 10: 'Estudiante (Escuela/Instituto)',
    11: 'Abogado', 12: 'Programador', 13: 'Jubilado', 14: 'Ventas / Marketing',
    15: 'Científico', 16: 'Autónomo', 17: 'Técnico / Ingeniero', 18: 'Artesano / Oficio manual',
    19: 'Desempleado', 20: 'Escritor'
}
df_users_mod = df_users.copy()
df_users_mod['occupation'] = df_users_mod['occupation'].map(occupation_map).astype('string')

age_groups = {
    1: (17, 'A'),
    18: (21, 'B'),
    25: (30, 'C'),
    35: (40, 'D'),
    45: (47, 'E'),
    50: (52, 'F'),
    56: (60, 'G'),
}
df_users_mod['age_group_mean'] = df_users_mod['age'].map(lambda v: age_groups.get(v, (None, None))[0])
df_users_mod['age_group_letter'] = df_users_mod['age'].map(lambda v: age_groups.get(v, (None, None))[1])

zip_state_ranges = [
    (350, 369, 'AL'), (995, 999, 'AK'), (850, 865, 'AZ'), (716, 729, 'AR'),
    (900, 961, 'CA'), (800, 816, 'CO'), (60, 69, 'CT'), (197, 199, 'DE'),
    (200, 200, 'DC'), (202, 205, 'DC'), (569, 569, 'DC'), (320, 349, 'FL'),
    (300, 319, 'GA'), (967, 968, 'HI'), (832, 838, 'ID'), (600, 629, 'IL'),
    (460, 479, 'IN'), (500, 528, 'IA'), (660, 679, 'KS'), (400, 427, 'KY'),
    (700, 715, 'LA'), (39, 49, 'ME'), (206, 219, 'MD'), (10, 27, 'MA'),
    (480, 499, 'MI'), (550, 567, 'MN'), (386, 397, 'MS'), (630, 658, 'MO'),
    (590, 599, 'MT'), (680, 693, 'NE'), (889, 898, 'NV'), (30, 38, 'NH'),
    (70, 89, 'NJ'), (870, 884, 'NM'), (100, 149, 'NY'), (270, 289, 'NC'),
    (580, 588, 'ND'), (430, 459, 'OH'), (730, 749, 'OK'), (970, 979, 'OR'),
    (150, 196, 'PA'), (28, 29, 'RI'), (290, 299, 'SC'), (570, 577, 'SD'),
    (370, 385, 'TN'), (750, 799, 'TX'), (885, 885, 'TX'), (840, 847, 'UT'),
    (50, 59, 'VT'), (201, 201, 'VA'), (220, 246, 'VA'), (980, 994, 'WA'),
    (247, 268, 'WV'), (530, 549, 'WI'), (820, 831, 'WY'), (6, 9, 'PR'),
    (8, 8, 'VI'), (969, 969, 'GU')
]

def zip_to_state(zip_code):
    if not isinstance(zip_code, str):
        return None
    digits = ''.join(ch for ch in zip_code if ch.isdigit())
    if len(digits) < 3:
        return None
    prefix = int(digits[:3])
    for low, high, state in zip_state_ranges:
        if low <= prefix <= high:
            return state
    return None

df_users_mod['state'] = df_users_mod['zip'].apply(zip_to_state)
#df_users_mod = df_users_mod.drop(columns=['age', 'zip'])


In [59]:
df_users_mod

,userId,gender,age,occupation,zip,age_group_mean,age_group_letter,state
0,1,F,1,Estudiante (Escuela/Instituto),48067,17,A,MI
1,2,M,56,Autónomo,70072,60,G,LA
2,3,M,25,Científico,55117,30,C,MN
3,4,M,45,Ejecutivo / Gerente,02460,47,E,MA
4,5,M,25,Escritor,55455,30,C,MN
...,...,...,...,...,...,...,...,...
6035,6036,F,25,Científico,32603,30,C,FL
6036,6037,F,45,Académico / Educador,76006,47,E,TX
6037,6038,F,56,Académico / Educador,14706,60,G,NY
6038,6039,F,45,Otro / No especificado,01060,47,E,MA


In [60]:
df_users_mod.describe(include="all")

,userId,gender,age,occupation,zip,age_group_mean,age_group_letter,state
count,6040.000000,6040,6040.000000,6040,6040,6040.000000,6040,6030
unique,NaN,2,NaN,21,3439,NaN,7,53
top,NaN,M,NaN,Estudiante universitario / Postgrado,48104,NaN,C,CA
freq,NaN,4331,NaN,759,19,NaN,2096,1110
mean,3020.500000,NaN,30.639238,NaN,NaN,35.095861,NaN,NaN
std,1743.742145,NaN,12.895962,NaN,NaN,11.725806,NaN,NaN
min,1.000000,NaN,1.000000,NaN,NaN,17.000000,NaN,NaN
25%,1510.750000,NaN,25.000000,NaN,NaN,30.000000,NaN,NaN
50%,3020.500000,NaN,25.000000,NaN,NaN,30.000000,NaN,NaN
75%,4530.250000,NaN,35.000000,NaN,NaN,40.000000,NaN,NaN


Hay 10 registros que no tienen estado asignado, vamos a ver qué pasa

In [ ]:
df_users_mod[df_users_mod['state'].isna()]

,userId,gender,age,occupation,zip,age_group_mean,age_group_letter,state
2242,2243,M,45,Artista,86701,47,E,None
3333,3334,M,35,Otro / No especificado,09142,40,D,None
3380,3381,M,25,Otro / No especificado,96671,30,C,None
3576,3577,F,56,Otro / No especificado,00231,60,G,None
3652,3653,M,18,Científico,96661,21,B,None
4277,4278,F,45,Ejecutivo / Gerente,09094,47,E,None
4537,4538,M,25,Programador,0956456,30,C,None
5185,5186,M,35,Programador,96555,40,D,None
5530,5531,M,50,Técnico / Ingeniero,09056,52,F,None
5727,5728,F,35,Escritor,09824,40,D,None


Tras investigarlo, resulta que esos zips sí corresponden a Norteamérica pero en espacios militares controlados por el extranjero, por lo que decido mantener esos valores (además de que probablemente me descarte esta columna después)

In [65]:
df_users_mod.loc[df_users_mod['state'].isna(), 'state'] = 'Other'
df_users_mod[df_users_mod['state'].isna()]

,userId,gender,age,occupation,zip,age_group_mean,age_group_letter,state


Solucionado, para que no haya nulos

In [75]:
df_users_mod = df_users_mod.drop(columns=['age', 'zip'])

## Ratings

In [85]:
nombre_archivo = 'ratings.dat' 
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)

df_ratings = pd.read_csv(ruta_completa, sep='::', engine='python', header=None, names=['userId', 'filmId', 'rating', "timestamp"], encoding='latin-1')
print(len(df_ratings))

1000209


In [86]:
df_ratings

,userId,filmId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [87]:
df_ratings.describe(include="all")

,userId,filmId,rating,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


Otra vez, sin nulos, los ids son buenos y el rating también. Vamos a modificar el timestamp por terminar de dejarlo "bonito"

In [ ]:
df_ratings_mod = df_ratings.copy()
df_ratings_mod['date'] = pd.to_datetime(df_ratings_mod['timestamp'], unit='s').dt.strftime('%d/%m/%Y')
df_ratings_mod = df_ratings_mod.drop(columns=['timestamp'])

In [90]:
df_ratings_mod

,userId,filmId,rating,date
0,1,1193,5,31/12/2000
1,1,661,3,31/12/2000
2,1,914,3,31/12/2000
3,1,3408,4,31/12/2000
4,1,2355,5,06/01/2001
...,...,...,...,...
1000204,6040,1091,1,26/04/2000
1000205,6040,1094,5,25/04/2000
1000206,6040,562,5,25/04/2000
1000207,6040,1096,4,26/04/2000


## Guardado de dfs

In [91]:
# Guardar los DataFrames transformados
from pathlib import Path
out_dir = Path('datasets/Transformados')
out_dir.mkdir(parents=True, exist_ok=True)

dfs = {
    'movies_mod': df_movies_mod,
    'users_mod': df_users_mod,
    'ratings_mod': df_ratings,
}

for name, frame in dfs.items():
    frame.head(5).to_csv(out_dir / f'{name}_head.csv', index=False)
    frame.to_parquet(out_dir / f'{name}.parquet', index=False)

sorted(out_dir.iterdir())


[PosixPath('datasets/Transformados/movies_mod.parquet'),
 PosixPath('datasets/Transformados/movies_mod_head.csv'),
 PosixPath('datasets/Transformados/ratings_mod.parquet'),
 PosixPath('datasets/Transformados/ratings_mod_head.csv'),
 PosixPath('datasets/Transformados/users_mod.parquet'),
 PosixPath('datasets/Transformados/users_mod_head.csv')]

# Join

In [95]:
# Join ratings con movies y users
df_ratings_full = df_ratings_mod.merge(df_movies_mod, on='filmId', how='left')
df_ratings_full = df_ratings_full.merge(df_users_mod, on='userId', how='left')
print(len(df_ratings_full))


1000209


In [96]:
df_ratings_full

,userId,filmId,rating,date,film,genders,year,gender,occupation,age_group_mean,age_group_letter,state
0,1,1193,5,31/12/2000,One Flew Over the Cuckoo's Nest,[Drama],1975,F,Estudiante (Escuela/Instituto),17,A,MI
1,1,661,3,31/12/2000,James and the Giant Peach,"[Animation, Children's, Musical]",1996,F,Estudiante (Escuela/Instituto),17,A,MI
2,1,914,3,31/12/2000,My Fair Lady,"[Musical, Romance]",1964,F,Estudiante (Escuela/Instituto),17,A,MI
3,1,3408,4,31/12/2000,Erin Brockovich,[Drama],2000,F,Estudiante (Escuela/Instituto),17,A,MI
4,1,2355,5,06/01/2001,"Bug's Life, A","[Animation, Children's, Comedy]",1998,F,Estudiante (Escuela/Instituto),17,A,MI
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,26/04/2000,Weekend at Bernie's,[Comedy],1989,M,Médico / Sector salud,30,C,NY
1000205,6040,1094,5,25/04/2000,"Crying Game, The","[Drama, Romance, War]",1992,M,Médico / Sector salud,30,C,NY
1000206,6040,562,5,25/04/2000,Welcome to the Dollhouse,"[Comedy, Drama]",1995,M,Médico / Sector salud,30,C,NY
1000207,6040,1096,4,26/04/2000,Sophie's Choice,[Drama],1982,M,Médico / Sector salud,30,C,NY


In [97]:
df_ratings_full = df_ratings_full.drop(columns=['state'])
df_ratings_full.columns

Index(['userId', 'filmId', 'rating', 'date', 'film', 'genders', 'year',
       'gender', 'occupation', 'age_group_mean', 'age_group_letter'],
      dtype='object')

In [98]:
# Guardar df_ratings_full en datasets como parquet
from pathlib import Path
full_path = Path('datasets/df_ratings_full.parquet')
df_ratings_full.to_parquet(full_path, index=False)
full_path


PosixPath('datasets/df_ratings_full.parquet')

In [99]:
# Cargar el parquet con el join completo
df_ratings_full_loaded = pd.read_parquet('datasets/df_ratings_full.parquet')
len(df_ratings_full_loaded)

1000209